In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras import backend as K
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import pickle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from random import randrange, uniform
from time import time
import glob
import shutil
import itertools
from time import time

#ignore warning messages 
import warnings
warnings.filterwarnings('ignore') 

sns.set()

In [ ]:
def show(image, cmap):
    if cmap == 1:
        plt.imshow(image)
    else:
        plt.imshow(image, cmap='gray')
    plt.show()

# Read image RGB

In [ ]:
image_color = cv2.imread('ktaba.jpg', cv2.COLOR_BGR2RGB)
plt.imshow(image_color)
plt.show()

# Remove noise

In [ ]:
image_denoise = cv2.fastNlMeansDenoisingColored(image_color, None, 25, 25, 7, 25)
plt.imshow(image_denoise)
plt.show()

# Grayscale

In [ ]:
image_gray = cv2.cvtColor(image_denoise, cv2.COLOR_BGR2GRAY)
plt.imshow(image_gray, cmap='gray')
plt.show()

In [ ]:
image_binary = cv2.adaptiveThreshold(image_gray, 255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
plt.imshow(image_binary, cmap='gray')
plt.show()

# Thinning

In [ ]:
kernel = np.ones((4,4),np.uint8)
image_thin = cv2.erode(image_binary, kernel, iterations = 1)
plt.imshow(image_thin, cmap='gray')
plt.show()

# Character level segmentation

In [ ]:
mask = cv2.inRange(image_thin, 0, 0)
show(mask, 'gray')

In [ ]:
mask_color = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB)
show(mask_color, 1)

In [ ]:
gray = mask

# Apply adaptive threshold
thresh = cv2.adaptiveThreshold(gray,255, 1, 1, 11, 2)
thresh_color = cv2.cvtColor(thresh,cv2.COLOR_GRAY2BGR)

# apply some dilation and erosion to join the gaps
thresh = cv2.dilate(thresh,None,iterations = 3)
thresh = cv2.erode(thresh,None,iterations = 2)

# Find the contours
contours,hierarchy = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)

# For each contour, find the bounding rectangle and draw it

i = 0
for cnt in contours:
    x,y,w,h = cv2.boundingRect(cnt)
    roi = mask[y:y+h, x:x+w]
    cv2.rectangle(mask,(x,x+w),(y,y+h),(255,0,0),2)
    #cv2.imwrite(f"segment{i}", roi)
    
    #i += 1
    #cv2.rectangle(thresh_color,(x,y),(x+w,y+h),(0,255,0),2)

# Finally show the image
plt.imshow(mask)
plt.show()